In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
sns.set()
%config InlineBackend.figure_format = 'retina'

In [2]:
DATA = 'data'
allFiles = glob.glob(DATA + "/*.csv")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_)
    list_.append(df)
data = pd.concat(list_)

In [320]:
# we'll ignore retweets and non-english tweets
data = data.loc[(data.language == 'English') & (data.retweet == 0)]

In [106]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import words
import re
from tokenizer import tokenizer

In [199]:
T = tokenizer.TweetTokenizer(preserve_handles=True, 
                             preserve_hashes=True, 
                             preserve_case=False, 
                             preserve_url=False,
                             regularize=True)
en_stop = set(stopwords.words('english') + ['rt'])
p_stemmer = PorterStemmer()
en_words = set(w.lower() for w in words.words())

In [347]:
def lower(x): 
    return x.lower()
def delspecial(x):
    return x.replace('â', '')
def tokenize(x): 
    return T.tokenize(x)
def rm_stop(x): 
    return [j for j in x if not j in en_stop]
def stem(x): 
    return [p_stemmer.stem(j) for j in x]
def english(x):
    return list(filter(lambda w: w in en_words, x))
def rm_noalphanum(x):
    return list(filter(lambda x: re.search(r'[A-Za-z]', x), x))

In [348]:
def preprocess(text, transformations=[lower, delspecial, tokenize, rm_stop, rm_noalphanum]):
    for t in transformations:
        text = t(text)
    return text

### On The Right

In [349]:
text = data.loc[(data.account_category == 'RightTroll')].content
docs = [preprocess(d) for d in text.values]

KeyboardInterrupt: 

In [336]:
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)
print('Before:', len(dictionary))
dictionary.filter_extremes(no_below=10, no_above=.20)
print('After:', len(dictionary))

Before: 124445
After: 20196


In [337]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [338]:
from gensim.models import LdaModel
model = LdaModel(corpus=corpus,
                 id2word=dictionary,
                 num_topics=8,
                 iterations=100)

In [261]:
topicdocs = np.zeros((100000, 8))

In [262]:
for idoc, doc in enumerate(model[corpus]):
    for it in doc:
        topicdocs[idoc, it[0]] = it[1]

In [339]:
import pyLDAvis.gensim
import warnings
pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=FutureWarning) 

In [340]:
pdata = pyLDAvis.gensim.prepare(model, corpus, dictionary)

In [341]:
pdata

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.174138  0.016080       1        1  15.135963
2     -0.212934 -0.052724       2        1  13.866459
6     -0.062186 -0.140870       3        1  12.954274
5      0.139655 -0.117512       4        1  12.953004
4     -0.231270  0.111741       5        1  12.142340
7     -0.155129 -0.045220       6        1  11.598673
1      0.228009 -0.137632       7        1  11.279017
3      0.119716  0.366137       8        1  10.070269, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
179    Default  73896.000000        trump  73896.000000  30.0000  30.0000
1349   Default  31434.000000        video  31434.000000  29.0000  29.0000
604    Default  13150.000000  #demndebate  13150.000000  28.0000  28.0000
3729   Default  30983.000000     breaking  30983.000000  27.0000  27.0000
1013   Default  11614.000000       donald  11614.000000  26.0000  26.0000
168    Default  12170.000000    president  12170.000000  25.0000  25.0000
603    Default  11417.000000   #demdebate  11417.000000  24.0000  24.0000
1502   Default  10628.000000        house  10628.000000  23.0000  23.0000
290    Default   9919.000000        white   9919.000000  22.0000  22.0000
2166   Default   8818.000000        media   8818.000000  21.0000  21.0000
182    Default  10999.000000      hillary  10999.000000  20.0000  20.0000
467    Default   7222.000000         news   7222.000000  19.0000  19.0000
1455   Default   5831.000000    democrats   5831.000000  18.0000  18.0000
43     Default   8168.000000         like   8168.000000  17.0000  17.0000
1072   Default   4987.000000     american   4987.000000  16.0000  16.0000
1303   Default   5232.000000      trump's   5232.000000  15.0000  15.0000
2290   Default   5242.000000       russia   5242.000000  14.0000  14.0000
169    Default  12315.000000           us  12315.000000  13.0000  13.0000
2323   Default   5225.000000         fake   5225.000000  12.0000  12.0000
1256   Default   4720.000000        north   4720.000000  11.0000  11.0000
332    Default   5237.000000         best   5237.000000  10.0000  10.0000
676    Default   6565.000000      clinton   6565.000000   9.0000   9.0000
5038   Default  10493.000000       trumpâ  10493.000000   8.0000   8.0000
542    Default   5070.000000         live   5070.000000   7.0000   7.0000
756    Default   4498.000000        #tcot   4498.000000   6.0000   6.0000
5302   Default   4417.000000        korea   4417.000000   5.0000   5.0000
167    Default   6916.000000          one   6916.000000   4.0000   4.0000
15428  Default   4030.000000         #top   4030.000000   3.0000   3.0000
174    Default   6098.000000      liberal   6098.000000   2.0000   2.0000
541    Default   5798.000000          day   5798.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
2       Topic8   1347.779053          tom   1348.726074   2.2949  -5.4923
4373    Topic8   1341.175171        david   1342.121826   2.2949  -5.4972
1398    Topic8   1258.102661         youâ   1259.049561   2.2948  -5.5611
585     Topic8   1254.781372         fear   1255.728516   2.2948  -5.5638
7695    Topic8   1241.171753       maxine   1242.118286   2.2948  -5.5747
2719    Topic8   1152.204224       waters   1153.150635   2.2948  -5.6491
5336    Topic8   1142.485596    terrorism   1143.432373   2.2948  -5.6575
8290    Topic8   1136.256958     lawmaker   1137.204102   2.2947  -5.6630
929     Topic8   1116.353516      tonight   1117.301758   2.2947  -5.6807
2933    Topic8   1096.724243      message   1097.671143   2.2947  -5.6984
5049    Topic8   1085.354980       warren   1086.301636   2.2947  -5.7088
342     Topic8   1071.611694        happy   1072.558228   2.2947  -5.7216
650     Topic8   1069.711182       failed   1070.658203   2.2947  -5.7234
653     Topic